# Solar Panel Fault Detection using YOLO

This notebook implements a YOLO-based detector for identifying faults in solar panels using thermal imaging.

Key fault types detected:
- **Hotspots**: Single heated cells indicating issues like shading or soiling
- **Cracks**: Microcracks in solar cells caused by manufacturing stress or environmental factors
- **Shadings**: Cold spots caused by external objects blocking sunlight

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import glob
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Reshape, Concatenate
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Dataset Information

This notebook is designed to work with the Photovoltaic Module Dataset (PVMD), which contains:
- 1,000 thermal images of solar panels
- Images collected using DJI Mavic 3 Thermal drone
- Standardized to 512×512×3 dimensions
- Collected on September 5, 2024, at Tshwane University of Technology, South Africa

The dataset is publicly available in the Mendeley data repository.

## YOLO Detector Class Implementation

Below is the implementation of our Solar Panel YOLO Detector class.

In [ ]:
class SolarPanelYOLODetector:
    def __init__(self, data_dir):
        """
        Initialize the Solar Panel YOLO Detector
        
        Parameters:
        data_dir (str): Directory containing the YOLO dataset (images and labels)
        """
        self.data_dir = data_dir
        self.img_size = (416, 416)  # Standard YOLO input size
        self.classes = ['Hotspots', 'Cracks', 'Shadings']
        self.num_classes = len(self.classes)
        self.model = None
        
        # YOLO-specific parameters
        self.anchors = np.array([
            [10, 13], [16, 30], [33, 23],  # Small objects
            [30, 61], [62, 45], [59, 119],  # Medium objects
            [116, 90], [156, 198], [373, 326]  # Large objects
        ])
        self.num_anchors = self.anchors.shape[0]
        self.yolo_outputs = 3  # Number of YOLO output layers

In [ ]:
# Data Loading Methods
def parse_yolo_annotation(self, annotation_path):
    """
    Parse YOLO format annotation file
    
    Parameters:
    annotation_path (str): Path to the annotation file
    
    Returns:
    list: List of bounding boxes in the format [class_id, x, y, w, h]
    """
    boxes = []
    with open(annotation_path, 'r') as f:
        for line in f.readlines():
            data = line.strip().split(' ')
            if len(data) == 5:  # class_id, x_center, y_center, width, height
                class_id = int(data[0])
                x_center = float(data[1])
                y_center = float(data[2])
                width = float(data[3])
                height = float(data[4])
                boxes.append([class_id, x_center, y_center, width, height])
    return boxes

def load_yolo_dataset(self, split_ratio=0.8):
    """
    Load YOLO dataset from the given directory
    
    Parameters:
    split_ratio (float): Train/test split ratio
    
    Returns:
    tuple: (train_images, train_annotations, test_images, test_annotations)
    """
    print("Loading YOLO dataset...")
    
    # Find all image files
    image_paths = glob.glob(os.path.join(self.data_dir, "images", "*.jpg"))
    image_paths.extend(glob.glob(os.path.join(self.data_dir, "images", "*.png")))
    
    # Shuffle the image paths
    random.shuffle(image_paths)
    
    # Split into train and test sets
    split_index = int(len(image_paths) * split_ratio)
    train_images = image_paths[:split_index]
    test_images = image_paths[split_index:]
    
    # Get corresponding annotation files
    train_annotations = []
    for img_path in train_images:
        base_name = os.path.basename(img_path)
        name_without_ext = os.path.splitext(base_name)[0]
        annotation_path = os.path.join(self.data_dir, "labels", f"{name_without_ext}.txt")
        if os.path.exists(annotation_path):
            train_annotations.append(annotation_path)
        else:
            print(f"Warning: No annotation file for {img_path}")
    
    test_annotations = []
    for img_path in test_images:
        base_name = os.path.basename(img_path)
        name_without_ext = os.path.splitext(base_name)[0]
        annotation_path = os.path.join(self.data_dir, "labels", f"{name_without_ext}.txt")
        if os.path.exists(annotation_path):
            test_annotations.append(annotation_path)
        else:
            print(f"Warning: No annotation file for {img_path}")
    
    print(f"Dataset loaded: {len(train_images)} training images, {len(test_images)} testing images")
    return train_images, train_annotations, test_images, test_annotations

In [ ]:
# Preprocessing Methods
def preprocess_image(self, img_path):
    """
    Preprocess a single image for YOLO input
    
    Parameters:
    img_path (str): Path to the image file
    
    Returns:
    numpy.ndarray: Preprocessed image
    """
    # Read the image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    
    # Resize to YOLO input size
    img = cv2.resize(img, self.img_size)
    
    # Convert to RGB (OpenCV uses BGR by default)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Normalize pixel values to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    return img

def preprocess_batch(self, image_paths, annotation_paths):
    """
    Preprocess a batch of images and annotations for YOLO training
    
    Parameters:
    image_paths (list): List of image file paths
    annotation_paths (list): List of annotation file paths
    
    Returns:
    tuple: (images, yolo_outputs)
    """
    batch_size = len(image_paths)
    images = np.zeros((batch_size, self.img_size[0], self.img_size[1], 3), dtype=np.float32)
    
    # Output grid sizes for the 3 YOLO output layers
    grid_sizes = [self.img_size[0] // s for s in [32, 16, 8]]  # 13, 26, 52 for 416x416 input
    
    # Initialize YOLO outputs (one for each scale)
    y_true = [
        np.zeros((batch_size, grid_sizes[i], grid_sizes[i], 
                  self.num_anchors // 3, 5 + self.num_classes), dtype=np.float32)
        for i in range(self.yolo_outputs)
    ]
    
    for i, (img_path, anno_path) in enumerate(zip(image_paths, annotation_paths)):
        # Preprocess image
        images[i] = self.preprocess_image(img_path)
        
        # Parse annotation
        boxes = self.parse_yolo_annotation(anno_path)
        
        # Assign boxes to the appropriate output grid and anchor
        for box in boxes:
            class_id, x_center, y_center, width, height = box
            
            # Determine which anchor box and output grid this box belongs to
            # (This is a simplified version - actual YOLO would calculate IoU with anchors)
            box_area = width * height
            anchor_areas = self.anchors[:, 0] * self.anchors[:, 1] / (self.img_size[0] * self.img_size[1])
            anchor_index = np.argmin(np.abs(anchor_areas - box_area))
            
            # Determine which output grid
            output_index = anchor_index // 3
            anchor_in_layer = anchor_index % 3
            
            # Calculate grid cell coordinates
            grid_size = grid_sizes[output_index]
            grid_x = int(x_center * grid_size)
            grid_y = int(y_center * grid_size)
            
            # Ensure within bounds
            grid_x = max(0, min(grid_x, grid_size - 1))
            grid_y = max(0, min(grid_y, grid_size - 1))
            
            # Calculate target values
            tx = x_center * grid_size - grid_x  # Offset within the grid cell
            ty = y_center * grid_size - grid_y
            tw = np.log(width * self.img_size[0] / self.anchors[anchor_index, 0])
            th = np.log(height * self.img_size[1] / self.anchors[anchor_index, 1])
            
            # Set target values in the output grid
            y_true[output_index][i, grid_y, grid_x, anchor_in_layer, 0:4] = [tx, ty, tw, th]
            y_true[output_index][i, grid_y, grid_x, anchor_in_layer, 4] = 1  # Object confidence
            y_true[output_index][i, grid_y, grid_x, anchor_in_layer, 5 + int(class_id)] = 1  # Class one-hot
    
    return images, y_true

In [ ]:
# Model Building Methods
def build_yolo_model(self):
    """
    Build a simplified YOLO model for solar panel fault detection
    
    Returns:
    tensorflow.keras.models.Model: YOLO model
    """
    # Input layer
    input_tensor = Input(shape=(self.img_size[0], self.img_size[1], 3))
    
    # Backbone
    x = self._conv_block(input_tensor, 32, 3)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    
    x = self._conv_block(x, 64, 3)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    
    x = self._conv_block(x, 128, 3)
    x = self._conv_block(x, 64, 1)
    x = self._conv_block(x, 128, 3)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    
    x = self._conv_block(x, 256, 3)
    x = self._conv_block(x, 128, 1)
    x = self._conv_block(x, 256, 3)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    
    x = self._conv_block(x, 512, 3)
    x = self._conv_block(x, 256, 1)
    x = self._conv_block(x, 512, 3)
    x = self._conv_block(x, 256, 1)
    x = self._conv_block(x, 512, 3)
    route_1 = x  # Route for feature map 1
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    
    x = self._conv_block(x, 1024, 3)
    x = self._conv_block(x, 512, 1)
    x = self._conv_block(x, 1024, 3)
    x = self._conv_block(x, 512, 1)
    x = self._conv_block(x, 1024, 3)
    route_2 = x  # Route for feature map 2
    
    # YOLO head for large objects
    large_objects = self._yolo_head(route_2, 512, self.num_anchors // 3 * (5 + self.num_classes))
    
    # Upsample and concatenate for medium objects
    x = self._conv_block(route_2, 256, 1)
    x = tf.keras.layers.UpSampling2D(2)(x)
    x = Concatenate()([x, route_1])
    x = self._conv_block(x, 256, 1)
    x = self._conv_block(x, 512, 3)
    medium_objects = self._yolo_head(x, 256, self.num_anchors // 3 * (5 + self.num_classes))
    
    # Reshape outputs to match YOLO format
    large_grid_size = self.img_size[0] // 32
    medium_grid_size = self.img_size[0] // 16
    
    large_output = Reshape((large_grid_size, large_grid_size, 
                           self.num_anchors // 3, 5 + self.num_classes))(large_objects)
    medium_output = Reshape((medium_grid_size, medium_grid_size, 
                            self.num_anchors // 3, 5 + self.num_classes))(medium_objects)
    
    # Create model
    model = Model(inputs=input_tensor, outputs=[large_output, medium_output])
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=[self._yolo_loss, self._yolo_loss]
    )
    
    self.model = model
    return model

def _conv_block(self, x, filters, kernel_size, strides=1):
    """
    Create a convolution block with batch normalization and leaky ReLU
    """
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', 
               use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    return x

def _yolo_head(self, x, filters, output_filters):
    """
    Create a YOLO head for predictions
    """
    x = self._conv_block(x, filters, 3)
    x = Conv2D(output_filters, 1, padding='same')(x)
    return x

def _yolo_loss(self, y_true, y_pred):
    """
    Custom YOLO loss function
    
    This is a simplified version of the YOLO loss function.
    In a real implementation, you would calculate the full YOLO loss with:
    - Object confidence loss
    - No object confidence loss
    - Box coordinate loss
    - Class prediction loss
    """
    # Mask for cells that contain objects
    object_mask = y_true[..., 4:5]
    
    # Box coordinate loss (only for cells with objects)
    box_loss = tf.reduce_sum(tf.square(y_true[..., 0:4] - y_pred[..., 0:4]), axis=-1, keepdims=True)
    box_loss = object_mask * box_loss
    
    # Object confidence loss
    confidence_loss = tf.square(y_true[..., 4:5] - y_pred[..., 4:5])
    
    # Class prediction loss
    class_loss = object_mask * tf.reduce_sum(
        tf.square(y_true[..., 5:] - y_pred[..., 5:]), axis=-1, keepdims=True
    )
    
    # Combine losses
    total_loss = box_loss + confidence_loss + class_loss
    
    return tf.reduce_mean(total_loss)

In [ ]:
# Training Methods
def train_model(self, train_images, train_annotations, batch_size=8, epochs=50, validation_split=0.1):
    """
    Train the YOLO model
    
    Parameters:
    train_images (list): List of training image paths
    train_annotations (list): List of training annotation paths
    batch_size (int): Batch size for training
    epochs (int): Number of epochs to train
    validation_split (float): Fraction of data to use for validation
    
    Returns:
    tensorflow.keras.callbacks.History: Training history
    """
    if self.model is None:
        self.build_yolo_model()
    
    # Split for validation
    val_size = int(len(train_images) * validation_split)
    val_images = train_images[-val_size:]
    val_annotations = train_annotations[-val_size:]
    train_images = train_images[:-val_size]
    train_annotations = train_annotations[:-val_size]
    
    # Create data generators
    train_generator = self._batch_generator(train_images, train_annotations, batch_size)
    val_generator = self._batch_generator(val_images, val_annotations, batch_size)
    
    # Calculate steps per epoch
    steps_per_epoch = len(train_images) // batch_size
    validation_steps = len(val_images) // batch_size
    
    # Train model
    history = self.model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=validation_steps
    )
    
    return history

def _batch_generator(self, image_paths, annotation_paths, batch_size):
    """
    Generate batches of data for training
    """
    n = len(image_paths)
    while True:
        for i in range(0, n, batch_size):
            batch_images = image_paths[i:i+batch_size]
            batch_annotations = annotation_paths[i:i+batch_size]
            
            # Pad the last batch if necessary
            if len(batch_images) < batch_size:
                batch_images = batch_images + [batch_images[0]] * (batch_size - len(batch_images))
                batch_annotations = batch_annotations + [batch_annotations[0]] * (batch_size - len(batch_annotations))
            
            X, y = self.preprocess_batch(batch_images, batch_annotations)
            yield X, y

In [ ]:
# Detection Methods
def detect(self, img_path, confidence_threshold=0.5):
    """
    Detect solar panel faults in an image
    
    Parameters:
    img_path (str): Path to the image file
    confidence_threshold (float): Threshold for object detection confidence
    
    Returns:
    list: List of detected objects with class, confidence, and bounding box
    """
    if self.model is None:
        raise ValueError("Model is not trained. Call build_yolo_model() first.")
    
    # Preprocess image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    
    orig_height, orig_width = img.shape[:2]
    
    # Preprocess for input
    input_img = cv2.resize(img, self.img_size)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    input_img = input_img.astype(np.float32) / 255.0
    input_img = np.expand_dims(input_img, axis=0)
    
    # Predict
    outputs = self.model.predict(input_img)
    
    # Process outputs
    detections = []
    
    grid_sizes = [self.img_size[0] // s for s in [32, 16]]  # 13, 26 for 416x416 input
    
    for output_idx, output in enumerate(outputs):
        output = output[0]  # Remove batch dimension
        grid_size = grid_sizes[output_idx]
        
        for row in range(grid_size):
            for col in range(grid_size):
                for anchor_idx in range(self.num_anchors // 3):
                    # Extract data
                    tx, ty, tw, th = output[row, col, anchor_idx, 0:4]
                    confidence = output[row, col, anchor_idx, 4]
                    class_scores = output[row, col, anchor_idx, 5:]
                    
                    # Only process boxes with confidence above threshold
                    if confidence < confidence_threshold:
                        continue
                    
                    # Convert to absolute coordinates
                    anchor_idx_global = output_idx * 3 + anchor_idx
                    anchor_w, anchor_h = self.anchors[anchor_idx_global]
                    
                    x = (col + tx) / grid_size
                    y = (row + ty) / grid_size
                    w = np.exp(tw) * anchor_w / self.img_size[0]
                    h = np.exp(th) * anchor_h / self.img_size[1]
                    
                    # Convert to corners format
                    x1 = max(0, (x - w/2) * orig_width)
                    y1 = max(0, (y - h/2) * orig_height)
                    x2 = min(orig_width, (x + w/2) * orig_width)
                    y2 = min(orig_height, (y + h/2) * orig_height)
                    
                    # Get class with highest score
                    class_id = np.argmax(class_scores)
                    class_score = class_scores[class_id]
                    
                    # Store detection
                    detections.append({
                        'class_id': class_id,
                        'class_name': self.classes[class_id],
                        'confidence': float(confidence * class_score),
                        'box': [int(x1), int(y1), int(x2), int(y2)]
                    })
    
    # Apply non-maximum suppression
    return self._non_max_suppression(detections, 0.45)

def _non_max_suppression(self, detections, iou_threshold=0.45):
    """
    Apply non-maximum suppression to remove overlapping detections
    
    Parameters:
    detections (list): List of detections
    iou_threshold (float): IoU threshold for considering boxes as overlapping
    
    Returns:
    list: Filtered list of detections
    """
    if not detections:
        return []
    
    # Sort by confidence
    detections.sort(key=lambda x: x['confidence'], reverse=True)
    
    # Apply NMS
    filtered_detections = []
    while detections:
        current = detections.pop(0)
        filtered_detections.append(current)
        
        i = 0
        while i < len(detections):
            if current['class_id'] == detections[i]['class_id'] and \
               self._calculate_iou(current['box'], detections[i]['box']) > iou_threshold:
                detections.pop(i)
            else:
                i += 1
    
    return filtered_detections

def _calculate_iou(self, box1, box2):
    """
    Calculate IoU (Intersection over Union) between two boxes
    """
    # Box format: [x1, y1, x2, y2]
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    
    # Calculate intersection area
    xi_min = max(x1_min, x2_min)
    yi_min = max(y1_min, y2_min)
    xi_max = min(x1_max, x2_max)
    yi_max = min(y1_max, y2_max)
    
    if xi_max <= xi_min or yi_max <= yi_min:
        return 0.0  # No intersection
    
    intersection_area = (xi_max - xi_min) * (yi_max - yi_min)
    
    # Calculate union area
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - intersection_area
    
    return intersection_area / union_area

In [ ]:
# Visualization Methods
def visualize_detections(self, img_path, detections):
    """
    Visualize detections on an image
    
    Parameters:
    img_path (str): Path to the image file
    detections (list): List of detections
    """
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    ax = plt.gca()
    plt.imshow(img)
    
    colors = {
        'Hotspots': 'red',
        'Cracks': 'green',
        'Shadings': 'blue'
    }
    
    for det in detections:
        x1, y1, x2, y2 = det['box']
        w, h = x2 - x1, y2 - y1
        
        # Draw bounding box
        rect = Rectangle((x1, y1), w, h, linewidth=2, 
                       edgecolor=colors[det['class_name']], 
                       facecolor='none')
        ax.add_patch(rect)
        
        # Add label
        plt.text(x1, y1 - 5, 
                f"{det['class_name']} {det['confidence']:.2f}", 
                color='white', fontsize=12, 
                bbox={'facecolor': colors[det['class_name']], 'alpha': 0.7, 'pad': 2})
    
    plt.title(f"Solar Panel Fault Detection: {len(detections)} faults detected")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Thermal Image Processing Methods
def process_thermal_image(self, img_path):
    """
    Process a thermal image to enhance fault visibility
    
    Parameters:
    img_path (str): Path to the thermal image
    
    Returns:
    numpy.ndarray: Processed image
    """
    # Read the image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    
    # Convert to grayscale if not already
    if len(img.shape) > 2:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    # Apply false-color mapping for better visualization
    colored = cv2.applyColorMap(enhanced, cv2.COLORMAP_JET)
    
    return colored

def detect_and_analyze(self, img_path):
    """
    Detect faults and analyze them in a thermal image
    
    Parameters:
    img_path (str): Path to the image file
    """
    # Process thermal image
    processed_img = self.process_thermal_image(img_path)
    
    # Detect faults
    detections = self.detect(img_path)
    
    # Count detections by type
    counts = {'Hotspots': 0, 'Cracks': 0, 'Shadings': 0}
    for det in detections:
        counts[det['class_name']] += 1
    
    # Calculate severity scores
    hotspot_severity = self._calculate_hotspot_severity(processed_img, detections)
    crack_severity = self._calculate_crack_severity(processed_img, detections)
    shading_severity = self._calculate_shading_severity(processed_img, detections)
    
    # Visualize
    self.visualize_detections(img_path, detections)
    
    # Print analysis
    print("\nFault Analysis:")
    print(f"Hotspots: {counts['Hotspots']} detected (Severity: {hotspot_severity:.2f}%)")
    print(f"Cracks: {counts['Cracks']} detected (Severity: {crack_severity:.2f}%)")
    print(f"Shadings: {counts['Shadings']} detected (Severity: {shading_severity:.2f}%)")
    
    # Overall severity
    total_severity = (hotspot_severity + crack_severity + shading_severity) / 3
    print(f"\nOverall Panel Health: {100 - total_severity:.2f}%")
    
    if total_severity > 50:
        print("Recommendation: Immediate maintenance required")
    elif total_severity > 25:
        print("Recommendation: Schedule maintenance soon")
    else:
        print("Recommendation: Normal operation, no immediate action required")

In [ ]:
# Severity Calculation Methods
def _calculate_hotspot_severity(self, img, detections):
    """Calculate severity score for hotspots"""
    hotspots = [d for d in detections if d['class_name'] == 'Hotspots']
    if not hotspots:
        return 0
    
    # Calculate based on area and intensity
    severity = 0
    img_area = img.shape[0] * img.shape[1]
    
    for det in hotspots:
        x1, y1, x2, y2 = det['box']
        hotspot_area = (x2 - x1) * (y2 - y1)
        area_ratio = hotspot_area / img_area
        
        # Extract the hotspot region and calculate intensity
        roi = img[y1:y2, x1:x2]
        if roi.size > 0:
            # For thermal images, higher values (brighter) indicate hotter areas
            intensity = np.mean(roi) / 255.0
            severity += area_ratio * intensity * 100 * 2  # Weight factor
    
    return min(severity, 100)  # Cap at 100%

def _calculate_crack_severity(self, img, detections):
    """Calculate severity score for cracks"""
    cracks = [d for d in detections if d['class_name'] == 'Cracks']
    if not cracks:
        return 0
    
    # Calculate based on length and location
    severity = 0
    img_diagonal = np.sqrt(img.shape[0]**2 + img.shape[1]**2)
    
    for det in cracks:
        x1, y1, x2, y2 = det['box']
        # Estimate crack length as box diagonal
        crack_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        length_ratio = crack_length / img_diagonal
        
        # Location factor - cracks in center are more severe
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        distance_to_center = np.sqrt(
            (center_x - img.shape[1]/2)**2 + 
            (center_y - img.shape[0]/2)**2
        )
        distance_ratio = 1 - (distance_to_center / (img_diagonal/2))
        
        severity += length_ratio * distance_ratio * 100 * 1.5  # Weight factor
    
    return min(severity, 100)  # Cap at 100%

def _calculate_shading_severity(self, img, detections):
    """Calculate severity score for shadings"""
    shadings = [d for d in detections if d['class_name'] == 'Shadings']
    if not shadings:
        return 0
    
    # Calculate based on area coverage
    severity = 0
    img_area = img.shape[0] * img.shape[1]
    
    for det in shadings:
        x1, y1, x2, y2 = det['box']
        shading_area = (x2 - x1) * (y2 - y1)
        area_ratio = shading_area / img_area
        
        # Extract region to analyze temperature difference
        roi = img[y1:y2, x1:x2]
        if roi.size > 0:
            # For shading, calculate temperature difference (lower values in thermal images)
            avg_temp = np.mean(roi)
            avg_img_temp = np.mean(img)
            temp_diff_ratio = abs(avg_temp - avg_img_temp) / 255.0
            
            severity += area_ratio * temp_diff_ratio * 100
    
    return min(severity, 100)  # Cap at 100%

## Example Usage

Now let's demonstrate how to use our Solar Panel YOLO Detector with some example code.

In [ ]:
# Define the path to your dataset
data_dir = "./solar_panel_dataset"  # Update this to your dataset path

# Initialize the detector
detector = SolarPanelYOLODetector(data_dir)

# Load the dataset
train_images, train_annotations, test_images, test_annotations = detector.load_yolo_dataset(split_ratio=0.8)

# Build the model
model = detector.build_yolo_model()
model.summary()

In [ ]:
# Train the model (commented out by default as it can take a long time)
# Uncomment to train

# history = detector.train_model(
#     train_images, 
#     train_annotations, 
#     batch_size=8, 
#     epochs=50, 
#     validation_split=0.1
# )

# # Plot training history
# plt.figure(figsize=(12, 4))
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()

### Testing with Sample Images

To use this on your own images, you'll need to have a trained model. For demonstration purposes, we'll assume you have a pre-trained model ready.

In [ ]:
# Function to test on sample images
def test_on_sample(detector, image_path):
    print(f"Testing on image: {image_path}")
    
    # 1. Process the thermal image
    processed = detector.process_thermal_image(image_path)
    
    # Display the processed image
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(processed, cv2.COLOR_BGR2RGB))
    plt.title("Processed Thermal Image")
    plt.axis('off')
    plt.show()
    
    # 2. Detect and analyze faults
    detector.detect_and_analyze(image_path)

In [ ]:
# Test on a sample image (update the path to your test image)
# test_on_sample(detector, "./solar_panel_dataset/images/sample_hotspot_01.jpg")

## Conclusion

This notebook has implemented a YOLO-based solar panel fault detection system that can:

1. Process thermal images of solar panels
2. Detect three types of faults: hotspots, cracks, and shadings
3. Analyze the severity of detected faults
4. Provide maintenance recommendations based on fault severity

The model architecture used is a simplified version of YOLO that can be trained on a custom dataset of solar panel images. The system can be deployed as part of a maintenance workflow for solar installations to detect issues early and prevent power loss.